<a href="https://colab.research.google.com/github/AfifaMasood/AfifaMasood/blob/main/testing3_movienet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install opencv-python # Install the latest version of opencv-python
!pip install opencv-python-headless # Install the latest version of opencv-python-headless
!pip install  tqdm tf-models-official
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 107.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.4/615.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import os
from tensorflow import keras
from official.projects.movinet.modeling import movinet
from official.projects.movinet.modeling import movinet_model

CLASSES_LIST = ["Normal", "Shoplifting"]

# Number of frames required by the model
SEQUENCE_LENGTH = 8

def preprocess_frame(frame):
    frame = cv2.resize(frame, (256, 256))
    frame = frame / 255.0  # Normalize
    return frame.astype(np.float32)

import keras

def load_trained_model(model_path):
    """
    Loads the trained MoviNet model from TensorFlow SavedModel format using TFSMLayer.
    """
    try:
        model = keras.layers.TFSMLayer(model_path, call_endpoint="serving_default")
        print("Model loaded successfully using TFSMLayer!")
        return model
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

def frame_predict(video_path, model, output_video_path):
    video_capture = cv2.VideoCapture(video_path)

    if not video_capture.isOpened():
        print(f"Error: Unable to open video file at {video_path}")
        return

    frame_buffer = []  # Buffer to store 8 frames

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    frame_rate = video_capture.get(cv2.CAP_PROP_FPS)
    width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (width, height))

    if not out.isOpened():
        print(f"Error: Unable to open video writer for output video at {output_video_path}")
        return

    frame_count = 0

    while True:
        ret, frame = video_capture.read()
        if not ret:
            print("End of video or no frames to read.")
            break

        print(f"Processing frame {frame_count}")

        # Preprocess the frame
        processed_frame = preprocess_frame(frame)
        frame_buffer.append(processed_frame)  # Add frame to buffer

        # Keep buffer size fixed to 8
        if len(frame_buffer) > SEQUENCE_LENGTH:
            frame_buffer.pop(0)  # Remove oldest frame

        # Predict only when we have 8 frames
        if len(frame_buffer) == SEQUENCE_LENGTH:
            input_data = np.array(frame_buffer)  # Convert list to NumPy array
            input_data = np.expand_dims(input_data, axis=0)  # Add batch dimension: (1, 8, 256, 256, 3)

            # Predict using TFSMLayer
            predictions = model(input_data)
            predictions = list(predictions.values())[0].numpy()


            predicted_class = np.argmax(predictions)
            predicted_label = CLASSES_LIST[predicted_class]
            predicted_prob = max(predictions[0])

            print(f"Predicted: {predicted_label} ({predicted_prob:.2f})")

            # Annotate frame
            color = (0, 0, 255) if predicted_label == "Shoplifting" else (255, 255, 255)
            text = f"Prediction: {predicted_label} ({predicted_prob:.2f})"
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 1.5
            font_thickness = 2
            text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)
            text_x, text_y = 10, 40

            cv2.rectangle(frame, (text_x - 5, text_y - text_size[1] - 5),
                          (text_x + text_size[0] + 5, text_y + 5), (0, 0, 0), -1)
            cv2.putText(frame, text, (text_x, text_y), font, font_scale, (255, 255, 255), font_thickness)

        out.write(frame)
        frame_count += 1

    video_capture.release()
    out.release()


def main():
    """
    Main function to process a video and save the output.
    """
    video_path = '/content/drive/MyDrive/FYP Dataset/Shoplifting/Shoplifting-54.mp4'
    output_folder = '/content/drive/MyDrive/output_folder_new_movienet'

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Created output folder: {output_folder}")

    output_video_path = os.path.join(output_folder, 'Shoplifting-54_output.mp4')

    # Updated Model Path
    model_path = "/content/drive/MyDrive/MOVINET_SHOPLIFTING_CPU_withouth5"

    model = load_trained_model(model_path)
    if model:
        frame_predict(video_path, model, output_video_path)
        print(f"Output video saved at: {output_video_path}")

if __name__ == "__main__":
    main()


Model loaded successfully using TFSMLayer!
Processing frame 0
Processing frame 1
Processing frame 2
Processing frame 3
Processing frame 4
Processing frame 5
Processing frame 6
Processing frame 7
Predicted: Normal (1.31)
Processing frame 8
Predicted: Normal (1.72)
Processing frame 9
Predicted: Normal (2.46)
Processing frame 10
Predicted: Normal (2.55)
Processing frame 11
Predicted: Normal (1.72)
Processing frame 12
Predicted: Normal (0.44)
Processing frame 13
Predicted: Normal (0.11)
Processing frame 14
Predicted: Normal (1.29)
Processing frame 15
Predicted: Normal (1.67)
Processing frame 16
Predicted: Normal (1.35)
Processing frame 17
Predicted: Normal (1.02)
Processing frame 18
Predicted: Normal (1.02)
Processing frame 19
Predicted: Normal (0.69)
Processing frame 20
Predicted: Normal (1.29)
Processing frame 21
Predicted: Normal (1.86)
Processing frame 22
Predicted: Normal (1.56)
Processing frame 23
Predicted: Normal (1.53)
Processing frame 24
Predicted: Normal (1.15)
Processing frame 

In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import os
from tensorflow import keras
from official.projects.movinet.modeling import movinet
from official.projects.movinet.modeling import movinet_model

CLASSES_LIST = ["Normal", "Shoplifting"]
SEQUENCE_LENGTH = 3

def preprocess_frame(frame):
    frame = cv2.resize(frame, (256, 256))
    frame = frame / 255.0  # Normalize
    return frame.astype(np.float32)

def load_trained_model(model_path):
    try:
        model = keras.layers.TFSMLayer(model_path, call_endpoint="serving_default")
        print("Model loaded successfully using TFSMLayer!")
        return model
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

def process_video(video_path, model, output_video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break


        frame_resized = preprocess_frame(frame)
        frames.append(frame_resized)

        if len(frames) == SEQUENCE_LENGTH:
            input_tensor = np.array(frames).reshape(1, SEQUENCE_LENGTH, 256, 256, 3)
            prediction = model(input_tensor)

            if isinstance(prediction, dict):
                prediction = list(prediction.values())[0]  # Extract tensor values

            predicted_label = CLASSES_LIST[np.argmax(prediction)]
            predicted_prob = np.max(prediction)


            print(f"Predicted: {predicted_label} ({predicted_prob:.2f})")

            # Set color based on prediction
            color = (0, 0, 255) if predicted_label == "Shoplifting" else (255, 255, 255)
            text = f"Prediction: {predicted_label} ({predicted_prob:.2f})"
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 1.5
            font_thickness = 2
            text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)
            text_x, text_y = 10, 40

            cv2.rectangle(frame, (text_x - 5, text_y - text_size[1] - 5),
                          (text_x + text_size[0] + 5, text_y + 5), (0, 0, 0), -1)
            cv2.putText(frame, text, (text_x, text_y), font, font_scale, color, font_thickness)

            frames = []  # Reset frame buffer

        out.write(frame)

        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

def main():
    video_path = '/content/drive/MyDrive/FYP Dataset/Shoplifting/Shoplifting-2.mp4'
    output_folder = '/content/drive/MyDrive/output_folder_new_movienet'

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Created output folder: {output_folder}")

    output_video_path = os.path.join(output_folder, 'Shoplifting-2_output3.mp4')
    model_path = "/content/drive/MyDrive/MOVINET_SHOPLIFTING_CPU_withouth5"

    model = load_trained_model(model_path)
    if model:
        process_video(video_path, model, output_video_path)
        print(f"Output video saved at: {output_video_path}")

if __name__ == "__main__":
    main()